In [1]:
#//----------------------------
#//LIBRARIES
    #Math
import math
    #Numeric Python
import numpy as np
    #Pandas (dataframes)
import pandas as pd
    #datetime for fate manipulation
from datetime import date, datetime, timedelta  
    #Regex for advanced string matching
import re
    #for time related stuff
import time
    #json library
import json
    #Analyst tools
import sys
sys.path.append('../')
from analysts_tools.growth import *
    #Procurement tools
from procurement_lib import send_slack_notification
from procurement_lib import redash
from analysts_tools.redash_methods import *

import random
import time
import datetime

import seaborn as sns
import matplotlib.pyplot as plt


from sklearn.model_selection import train_test_split
from scipy.signal import savgol_filter
import statsmodels.formula.api as smf

In [2]:
df_sd = get_fresh_query_result("https://internal-redash.federate.frubana.com/",165803,'SeoGHWmDUaaBi7VXje1s9zYNiMD1VHQ1K1DYOxiF',{},20)

200


In [3]:
skus_sd = df_sd.card_id.unique()

In [4]:
df_margin = get_fresh_query_result("https://internal-redash.federate.frubana.com/",144521,'SeoGHWmDUaaBi7VXje1s9zYNiMD1VHQ1K1DYOxiF',{},20)
df_margin = df_margin.loc[df_margin['day'] == df_margin['day'].max()].copy()

200


In [5]:
query = """
-- 93058
-- Se toma como base el query 2718
WITH 
ful AS (
SELECT DISTINCT
    bo.order_id,
    --ffg.close_date
    bo.submit_date
    
FROM postgres_broadleaf_federate."broadleaf.blc_order" bo
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_fulfillment_group" bfg        ON bfg.order_id = bo.order_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_fulfillment_order" bfo        ON bfo.fulfillment_group_id = bfg.fulfillment_group_id
INNER JOIN postgres_broadleaf_federate."broadleaf.fb_fulfillment_group" ffg         ON ffg.fulfillment_group_id = bfg.fulfillment_group_id
INNER JOIN postgres_broadleaf_federate."broadleaf.fb_order" fo                      ON fo.order_id = bo.order_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_order_payment" bop            ON bop.order_id = bo.order_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_site" s                       ON s.site_id = bo.site_disc
LEFT JOIN  postgres_broadleaf_federate."broadleaf.fb_order_type" fot                ON fot.fb_order_type_id=fo.fb_order_type_id

WHERE fo.fb_order_status_id IN (1,6,7,8)
    AND bo.submit_date >= DATE_TRUNC('month', date(getdate()) - interval '1 month')
    AND bo.submit_date < DATE_TRUNC('month', date(getdate()) - interval '0 month')
    AND bo.order_status = 'SUBMITTED'
    AND bfo.status NOT IN ('ARCHIVED','CANCELLED')
    AND bop.archived = 'N'
    AND (fot.name IS NULL OR fot.name <> 'REFUND')
)

SELECT DISTINCT
    s.site_identifier_value AS city,
    --DATE(bo.submit_date) AS submit_date,
    --bo.customer_id,
    bs2.name,
    bs.addl_product_id as padre_sku_id,
    --bcat2.name AS category,
    --bcat.name AS subcat,
    --MIN(bo.submit_date) AS min_date,
    --COUNT(DISTINCT bo.order_id) as orders,
    CASE WHEN s.site_identifier_value = 'CMX' then SUM ( (boi.quantity * foi.step_unit * boi.sale_price)*1.0/19.65)
        WHEN s.site_identifier_value = 'GDL' then SUM ( (boi.quantity * foi.step_unit * boi.sale_price)*1.0/19.65)
        WHEN s.site_identifier_value = 'PBC' then SUM ( (boi.quantity * foi.step_unit * boi.sale_price)*1.0/19.65)
        WHEN s.site_identifier_value = 'MTY' then SUM ( (boi.quantity * foi.step_unit * boi.sale_price)*1.0/19.65)
        WHEN s.site_identifier_value = 'SPO' then SUM ( (boi.quantity * foi.step_unit * boi.sale_price)*1.0/4.75)
        WHEN s.site_identifier_value = 'BHZ' then SUM ( (boi.quantity * foi.step_unit * boi.sale_price)*1.0/4.75)
        WHEN s.site_identifier_value = 'CWB' then SUM ( (boi.quantity * foi.step_unit * boi.sale_price)*1.0/4.75)
        WHEN s.site_identifier_value = 'VCP' then SUM ( (boi.quantity * foi.step_unit * boi.sale_price)*1.0/4.75)
    ELSE SUM ( (boi.quantity * foi.step_unit * boi.sale_price)*1.0/3776) END AS gmv_usd,
    SUM(boi.quantity* foi.step_unit) as cant

FROM postgres_broadleaf_federate."broadleaf.blc_order" bo
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_site"                     s       ON s.site_id = bo.site_disc
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_order_item"               boi     ON boi.order_id=bo.order_id
INNER JOIN postgres_broadleaf_federate."broadleaf.fb_order_item"                foi     ON boi.order_item_id= foi.order_item_id
INNER JOIN postgres_broadleaf_federate."broadleaf.fb_order"                     fo      ON fo.order_id = bo.order_id
INNER JOIN ful                                                                          ON ful.order_id = bo.order_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_discrete_order_item"      bdoi    ON bdoi.order_item_id = boi.order_item_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_sku"                      bs      ON bs.sku_id = bdoi.sku_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_product"                  bp      ON bs.addl_product_id = bp.product_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_sku"                      bs2     ON bs2.sku_id = bp.default_sku_id --Conectar la tarjeta con el sku que guarda la info
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_category"                 bcat    ON bcat.category_id = bp.default_category_id 
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_category_xref"            bcx     ON bcx.sub_category_id = bp.default_category_id AND bcx.archived='N' AND bcx.sndbx_tier is NULL
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_category"                 bcat2   ON bcx.category_id = bcat2.category_id
--INNER JOIN postgres_broadleaf_federate."broadleaf.blc_order_item_price_dtl"     boipd   ON boipd.order_item_id=boi.order_item_id
--INNER JOIN postgres_broadleaf_federate."broadleaf.blc_order_item_dtl_adj"       boida   ON boida.order_item_price_dtl_id=boipd.order_item_price_dtl_id

WHERE fb_order_status_id IN (1,6,7,8)
    AND bo.order_status = 'SUBMITTED'
   -- FIX SUPER DESCUENTOS
    AND bcat2.category_id not in ('110873','-1000','100768','100765','100815') --ids de super descuentos en cada país
    AND bcat.name <> 'Oferton Frubana' AND bcat2.name <> 'Oferton Frubana'
    AND bs.addl_product_id IN {skus}    
GROUP BY 1,2,3
""".format(skus=tuple(skus_sd))

data_ventas_aux = read_connection_data_warehouse.runQuery(query)
data_ventas_aux[["gmv_usd","cant"]] = data_ventas_aux[["gmv_usd","cant"]].astype(float)

In [6]:
df = pd.merge(df_sd, df_margin[['region','margin','id_tarjeta']], left_on=['card_id','city'], right_on=['id_tarjeta','region'], how='inner')
df = pd.merge(df, data_ventas_aux[['city','gmv_usd','padre_sku_id']], left_on=['card_id','city'], right_on=['padre_sku_id','city'], how='inner')

In [7]:
df[df.offer_id == -14510917]

,city,offer_id,discount,value_discount,card_id,name,cat,subcat,region,margin,id_tarjeta,gmv_usd,padre_sku_id


In [8]:
weighted_margin = df.groupby(['city','offer_id','discount','value_discount']).apply(lambda x: (x['margin'] * x['gmv_usd']).sum() / x['gmv_usd'].sum()).reset_index()
weighted_margin.columns = ['city','offer_id','discount','value_discount','weighted_margin']
weighted_margin['weighted_margin'] = weighted_margin['weighted_margin'].apply(lambda x: round(x*100, 2))

In [9]:
weighted_margin

,city,offer_id,discount,value_discount,weighted_margin
0,BAQ,-14520562,20230701_BAQ_other_piloto_NB_quesos_W26,11,15.69
1,BAQ,-14520561,20230701_BAQ_other_piloto_NB_leches_W26,12,3.59
2,BAQ,-14520560,20230701_BAQ_other_piloto_NB_huevos_W26,7,6.60
3,BAQ,-14520559,20230701_BAQ_other_piloto_NB_derivadoslacteos_W26,10,11.46
4,BAQ,-14520558,20230701_BAQ_other_piloto_NB_cremas_W26,6,15.57
...,...,...,...,...,...
1800,VCP,-14521751,20230701_VCP_other_piloto_NB_destilados_W26,5,18.12
1801,VCP,-14521750,20230701_VCP_other_piloto_NB_sucosechas_W26,8,19.73
1802,VCP,-14521749,20230701_VCP_other_piloto_NB_refrigeranteseene...,6,8.86
1803,VCP,-14521748,20230701_VCP_other_piloto_NB_cervejas_W26,4,8.37


In [10]:
weighted_margin.to_excel("sd_margin.xlsx")